__Note__: The code used here was heavily inspired by the AI for Medicine Specialization Course 1 Week three programming assignments: [link](https://www.coursera.org/learn/ai-for-medical-diagnosis)

In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import json
import os
from pprint import pprint
from collections import OrderedDict
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
from sys import getsizeof
from tqdm import tqdm

### Checking BRATS_001 nifti MRI and mask files before and after splitting

### 1- Before

#### MRI file

In [2]:
mri_1 = nib.load("../../datasets/Task01_BrainTumour/train/images/BRATS_001.nii.gz")

In [3]:
type(mri_1)

nibabel.nifti1.Nifti1Image

In [4]:
# Read Nifi Image object as numpy file
mri_1 = mri_1.get_fdata()

In [5]:
type(mri_1)

numpy.ndarray

In [6]:
mri_1.dtype

dtype('float64')

In [7]:
mri_1.shape

(240, 240, 155, 4)

#### Mask file

In [8]:
mri_1_mask = nib.load("../../datasets/Task01_BrainTumour/train/masks/BRATS_001.nii.gz")

In [9]:
mri_1_mask = mri_1_mask.get_fdata()

In [10]:
mri_1_mask.dtype

dtype('float64')

In [11]:
np.unique(mri_1_mask)

array([0., 1., 2., 3.])

In [12]:
mri_1_mask = mri_1_mask.astype(np.float32)

In [13]:
mri_1_mask.shape

(240, 240, 155)

### Visualizing original nifti MRI file and Mask file

In [14]:
classes_dict = {
    'Background': 0,
    'Edema': 1,
    'Non-enhancing tumor': 2,
    'Enhancing tumor': 3 
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Edema', 'Non-enhancing tumor', 'Enhancing tumor', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=mri_1.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.imshow(mri_1[:, :, layer, channel], cmap='gray');
    plt.title("MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        mask = mri_1_mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(mri_1_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: "FLAIR" Fluid-attenuated inversion recovery
#  Channel 1: "T1w" T1-weighted
#  Channel 2: "t1gd" T1-weighted with gadolinium contrast enhancement
#  Channel 3: "T2w" T2-weighted    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 3))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### 2- After

#### MRI file

In [15]:
mri_1_after = nib.load("../../datasets/Task01_BrainTumour_240x240x160x4/train/images/BRATS_001.nii.gz")

In [16]:
type(mri_1_after)

nibabel.nifti1.Nifti1Image

In [17]:
# Read Nifi Image object as numpy file
mri_1_after = mri_1_after.get_fdata().astype(np.float32)

In [18]:
type(mri_1_after)

numpy.ndarray

In [19]:
mri_1_after.dtype

dtype('float32')

In [20]:
mri_1_after.shape

(240, 240, 160, 4)

#### Mask file

In [21]:
mri_1_mask_after = nib.load("../../datasets/Task01_BrainTumour_240x240x160x4/train/masks/BRATS_001.nii.gz")

In [22]:
mri_1_mask_after = mri_1_mask_after.get_fdata().astype(np.float32)

In [23]:
mri_1_mask_after.dtype

dtype('float32')

In [24]:
np.unique(mri_1_mask_after)

array([0., 1., 2., 3.], dtype=float32)

In [25]:
mri_1_mask_after = mri_1_mask_after.astype(np.float32)

In [26]:
mri_1_mask_after.shape

(240, 240, 160)

### Visualizing nifti MRI file and Mask file

In [27]:
classes_dict = {
    'Background': 0,
    'Edema': 1,
    'Non-enhancing tumor': 2,
    'Enhancing tumor': 3 
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Edema', 'Non-enhancing tumor', 'Enhancing tumor', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=mri_1_after.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.imshow(mri_1_after[:, :, layer, channel], cmap='gray');
    plt.title("MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        mask = mri_1_mask_after[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(mri_1_mask_after[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: "FLAIR" Fluid-attenuated inversion recovery
#  Channel 1: "T1w" T1-weighted
#  Channel 2: "t1gd" T1-weighted with gadolinium contrast enhancement
#  Channel 3: "T2w" T2-weighted    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 3))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### Checking BRATS_002 nifti MRI and mask files before and after splitting

### 1- Before

#### MRI file

In [28]:
mri_2 = nib.load("../../datasets/Task01_BrainTumour/train/images/BRATS_002.nii.gz")

In [29]:
type(mri_2)

nibabel.nifti1.Nifti1Image

In [30]:
# Read Nifi Image object as numpy file
mri_2 = mri_2.get_fdata()

In [31]:
type(mri_2)

numpy.ndarray

In [32]:
mri_2.dtype

dtype('float64')

In [33]:
mri_2.shape

(240, 240, 155, 4)

#### Mask file

In [34]:
mri_2_mask = nib.load("../../datasets/Task01_BrainTumour/train/masks/BRATS_002.nii.gz")

In [35]:
mri_2_mask = mri_2_mask.get_fdata()

In [36]:
mri_2_mask.dtype

dtype('float64')

In [37]:
np.unique(mri_2_mask)

array([0., 1., 2., 3.])

In [38]:
mri_2_mask = mri_2_mask.astype(np.float32)

In [39]:
mri_2_mask.shape

(240, 240, 155)

### Visualizing original nifti MRI file and Mask file

In [40]:
classes_dict = {
    'Background': 0,
    'Edema': 1,
    'Non-enhancing tumor': 2,
    'Enhancing tumor': 3 
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Edema', 'Non-enhancing tumor', 'Enhancing tumor', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=mri_2.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.imshow(mri_2[:, :, layer, channel], cmap='gray');
    plt.title("MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        mask = mri_2_mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(mri_2_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: "FLAIR" Fluid-attenuated inversion recovery
#  Channel 1: "T1w" T1-weighted
#  Channel 2: "t1gd" T1-weighted with gadolinium contrast enhancement
#  Channel 3: "T2w" T2-weighted    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 3))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### 2- After

#### MRI file

In [41]:
mri_2_after = nib.load("../../datasets/Task01_BrainTumour_240x240x160x4/train/images/BRATS_002.nii.gz")

In [42]:
type(mri_2_after)

nibabel.nifti1.Nifti1Image

In [43]:
# Read Nifi Image object as numpy file
mri_2_after = mri_2_after.get_fdata().astype(np.float32)

In [44]:
type(mri_2_after)

numpy.ndarray

In [45]:
mri_2_after.dtype

dtype('float32')

In [46]:
mri_2_after.shape

(240, 240, 160, 4)

#### Mask file

In [47]:
mri_2_mask_after = nib.load("../../datasets/Task01_BrainTumour_240x240x160x4/train/masks/BRATS_002.nii.gz")

In [48]:
mri_2_mask_after = mri_2_mask_after.get_fdata().astype(np.float32)

In [49]:
mri_2_mask_after.dtype

dtype('float32')

In [50]:
np.unique(mri_2_mask_after)

array([0., 1., 2., 3.], dtype=float32)

In [51]:
mri_2_mask_after = mri_2_mask_after.astype(np.float32)

In [52]:
mri_2_mask_after.shape

(240, 240, 160)

### Visualizing nifti MRI file and Mask file

In [53]:
classes_dict = {
    'Background': 0,
    'Edema': 1,
    'Non-enhancing tumor': 2,
    'Enhancing tumor': 3 
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Edema', 'Non-enhancing tumor', 'Enhancing tumor', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=mri_2_after.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.imshow(mri_2_after[:, :, layer, channel], cmap='gray');
    plt.title("MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        mask = mri_2_mask_after[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(mri_2_mask_after[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: "FLAIR" Fluid-attenuated inversion recovery
#  Channel 1: "T1w" T1-weighted
#  Channel 2: "t1gd" T1-weighted with gadolinium contrast enhancement
#  Channel 3: "T2w" T2-weighted    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 3))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…